## [21]: データを読み込んで把握

In [2]:
import pandas as pd

In [ ]:
uselog = pd.read_csv("use_log.csv") # ジムの利用履歴データ、2018年4月～2019年3月
print(len(uselog))
uselog.head()

197428


,log_id,customer_id,usedate
0,L00000049012330,AS009373,2018-04-01
1,L00000049012331,AS015315,2018-04-01
2,L00000049012332,AS040841,2018-04-01
3,L00000049012333,AS046594,2018-04-01
4,L00000049012334,AS073285,2018-04-01


In [4]:
customer = pd.read_csv("customer_master.csv") # 2019年3月末時点での会員データ
print(len(customer))
customer.head()

4192


,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0


In [8]:
class_master = pd.read_csv("class_master.csv") # 会員区分データ
print(len(class_master))
class_master.head()

3


,class,class_name,price
0,C01,オールタイム,10500
1,C02,デイタイム,7500
2,C03,ナイト,6000


In [6]:
campaign_master = pd.read_csv("campaign_master.csv") # キャンペーン区分データ
print(len(campaign_master))
campaign_master.head()

3


,campaign_id,campaign_name
0,CA1,通常
1,CA2,入会費半額
2,CA3,入会費無料


## [22]: 顧客データを整形

In [ ]:
customer_join = pd.merge(customer, class_master, on = "class", how = "left") # 会員区分マスタと結合
customer_join = pd.merge(customer_join, campaign_master, on = "campaign_id", how = "left") # キャンペーン区分マスタと結合
customer_join.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,オールタイム,10500,通常
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,オールタイム,10500,通常
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,オールタイム,10500,通常
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,オールタイム,10500,通常
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0,ナイト,6000,通常


In [12]:
print(len(customer))
print(len(customer_join))

4192
4192


In [13]:
customer_join.isnull().sum() # 欠損値の確認

customer_id         0
name                0
class               0
gender              0
start_date          0
end_date         2842
campaign_id         0
is_deleted          0
class_name          0
price               0
campaign_name       0
dtype: int64

## [23]: 顧客データの基礎集計

In [15]:
customer_join.groupby("class_name").count()["customer_id"] # 会員区分ごとの会員数を集計

class_name
オールタイム    2045
デイタイム     1019
ナイト       1128
Name: customer_id, dtype: int64

In [17]:
customer_join.groupby("campaign_name").count()["customer_id"] # キャンペーン区分ごとの会員数を集計

campaign_name
入会費半額     650
入会費無料     492
通常       3050
Name: customer_id, dtype: int64

In [19]:
customer_join.groupby("gender").count()["customer_id"] # 性別ごとの会員数を集計

gender
F    1983
M    2209
Name: customer_id, dtype: int64

In [ ]:
customer_join.groupby("is_deleted").count()["customer_id"] # 退会ユーザー数を集計

is_deleted
0    2842
1    1350
Name: customer_id, dtype: int64

In [24]:
customer_join["start_date"] = pd.to_datetime(customer_join["start_date"]) # 開始日をdatetime型に変換
customer_start = customer_join.loc[customer_join["start_date"] > pd.to_datetime("20180401")] # 開始日が2018年4月1日以降の会員を抽出
print(len(customer_start))

1361


## [24]: 最新顧客データの基礎集計

In [25]:
customer_join["end_date"] = pd.to_datetime(customer_join["end_date"]) # 終了日をdatetime型に変換
customer_newer = customer_join.loc[(customer_join["end_date"] >= pd.to_datetime("20190331"))|(customer_join["end_date"].isna())] # 終了日が2019年3月31日以降の会員を抽出
print(len(customer_newer))
customer_newer["end_date"].unique() # 終了日がNaTの会員を抽出

2953


<DatetimeArray>
['NaT', '2019-03-31 00:00:00']
Length: 2, dtype: datetime64[ns]

- ① customer_join["end_date"] = pd.to_datetime(customer_join["end_date"])
  - customer_join というデータフレームの end_date 列（退会日など）を、文字列 → 日付型（datetime）に変換しています。
  - これにより、後で日付の大小比較などができるようになります。
<br><br>
- ② customer_newer = customer_join.loc[(customer_join["end_date"] >= pd.to_datetime("20190331")) | (customer_join["end_date"].isna())]
  - この部分は、**2019年3月31日以降に退会した人、または退会していない人（end_date が欠損値）**を抽出しています。
- customer_join["end_date"] >= pd.to_datetime("20190331")
  - → 2019年3月31日 以降の退会日（＝比較的新しい退会者）
- customer_join["end_date"].isna()
  - → end_date が NaN（= まだ退会していない顧客）
- | は または（OR） という意味です。
- つまり → 「最近まで在籍している、またはまだ辞めていない顧客」だけを抽出している。
<br><br>
- customer_newer["end_date"].unique()
  - 抽出されたデータの中で、end_date の**ユニークな値（重複なし一覧）**を表示しています。
  - つまり、どんな退会日が含まれているのかを確認するための処理です。

In [27]:
customer_newer.groupby("class_name").count()["customer_id"] # 会員区分ごとの会員数を集計

class_name
オールタイム    1444
デイタイム      696
ナイト        813
Name: customer_id, dtype: int64

In [29]:
customer_newer.groupby("campaign_name").count()["customer_id"] # キャンペーン区分ごとの会員数を集計

campaign_name
入会費半額     311
入会費無料     242
通常       2400
Name: customer_id, dtype: int64

In [31]:
customer_newer.groupby("gender").count()["customer_id"] # 性別ごとの会員数を集計

gender
F    1400
M    1553
Name: customer_id, dtype: int64